In [0]:
import sys
sys.path.append("../../")

In [0]:
from pyspark.sql.functions import *
from bussiness.customer import bv_bridge_agent_current_schema

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'
partition_date_fmt = f"{partition_date[:4]}-{partition_date[4:6]}-{partition_date[6:]}"

In [0]:
hub_cust_table = "ctl_central_published.sc_sv_dv.hub_customer_daily"
link_cust_pol_table = "ctl_central_published.sc_sv_dv.link_customer_policy"
link_pol_agent_table = "ctl_central_published.sc_sv_dv.link_policy_agent"

output_table = "ctl_central_published.sc_sv_bv_customer.bridge_agent_current"

# Customer

In [0]:
today_cust = spark.sql(f"""
                        select * 
                        from {hub_cust_table}
                        where {partition_name} = '{partition_date_fmt}'
                       """)
today_cust.display()

# Link customer policy

In [0]:
link_cust_pol = spark.sql(f"""
                        select * 
                        from {link_cust_pol_table}
                       """)
link_cust_pol.display()

In [0]:
today_cust_pol = today_cust.alias("t").join(link_cust_pol.alias('l'), on="customer_id", how="left")\
    .select("t.customer_id",
            "t.partition_date",
            "l.policy_number",)
today_cust_pol.display()

# Link cust agent

In [0]:
link_pol_agent = spark.sql(f"""
                        select * 
                        from {link_pol_agent_table}
                       """)
link_pol_agent.display()

In [0]:
today_cust_pol_agent = today_cust_pol.alias("t").join(link_pol_agent.alias('l'), on="policy_number", how="left")\
    .select("t.customer_id",
            "t.partition_date",
            "t.policy_number",
            "l.license_number",
    ).drop("policy_number")\
    .dropDuplicates()
today_cust_pol_agent.display()

# Save to table

In [0]:
if not spark.catalog.tableExists(output_table):
    bv_bridge_agent_current_schema = bv_bridge_agent_current_schema()
    (
        spark.createDataFrame([], bv_bridge_agent_current_schema)
        .write
        .format("delta")
        .partitionBy(partition_name)
        .saveAsTable(output_table)
    )

In [0]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

today_cust_pol_agent.write.mode("overwrite").saveAsTable(output_table)